# **Projek Akhir Praktikum Data Science**

> Analisis Sentimen Mengenai Vaksin COVID-19 Di Indonesia Menggunakan Metode Naive Bayes Classifier dan NLP Pada Sosial Media Twitter

**Oleh Kelompok 2 :**
1. Hazlan Muhammad Qodri (123190080)
2. Elisia Dwi Rahayu (123190062)
3. Shania Septika Inayasari (123190055)

**Penjelasan Projek :**

Adapun pada penelitian menekankan kepada sentimen masyarakat terhadap mengenai vaksin COVID-19. Proses analisisnya akan dilakukan berdasarkan tweet yang menyertakan tagar vaksin dan pencarian di twitter dengan keyword vaksin covid 19.

# **1. Scraping Data from Twitter**

In [ ]:
%pip install tweepy

In [2]:
import tweepy
import csv
import os
from datetime import timedelta, datetime
import pandas as pd

#### Secret Env Variable

In [4]:
consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_key_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')
bearer_token = os.environ.get('bearer_token')

In [6]:
# token for twitter developers
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret,
)

def scrap_tweets(search_words, date_since, date_until):
    print(search_words, date_since, date_until)

# Date Config
today = datetime.today().strftime("%Y-%m-%d")
last_week = datetime.today() - timedelta(7)
last_week = last_week.strftime("%Y-%m-%d")

# Searching Word
search_words = "covid indonesia OR kasus covid OR #kasuscovid OR #vaksin OR #waspadacovid OR #vaksincovid19 OR #vaksincovid OR #vaksingratis OR vaksin OR vaksin gratis"
date_since = last_week
date_until = today

scrap_tweets(search_words, date_since, date_until)


covid indonesia OR kasus covid OR #kasuscovid OR #vaksin OR #waspadacovid OR #vaksincovid19 OR #vaksincovid OR #vaksingratis OR vaksin OR vaksin gratis 2022-11-09 2022-11-16
